In [1]:
# preparation (calculate hash values of code units)
import os
import inspect
import hashlib
import time

fList = list()


def print_py_file_paths(folder_path):
    fList.clear()
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.py') and "Test" not in root:
                file_path = os.path.join(root, file)
                fList.append(file_path)


def hash_function(source_code):
    """
    Takes in the source code of a function as a string and returns the SHA-256 hash
    of the source code as a hexadecimal string.
    """
    hash_object = hashlib.sha256(source_code.encode())
    hex_dig = hash_object.hexdigest()
    return hex_dig


def hash_functions_in_file(file_path):
    """
    Takes in a file path and returns a dictionary mapping function names to their
    corresponding SHA-256 hash of source code.
    """
    function_hashes = {}

    with open(file_path, "rt", encoding="UTF-8") as f:
        source_code = f.read()
        file_name = file_path.split("\\")[-1]
        # Split source code by "def " to get each function's source code
        function_sources = source_code.split("def ")[1:]

        for function_source in function_sources:
            # Get the function name and source code
            function_name = function_source.split("(")[0]
            source_code = "def " + function_source

            # Hash the source code and store in the dictionary
            function_hash = hash_function(source_code)
            function_hashes[file_name + "\\" + function_name + "()"] = function_hash

    return function_hashes

dic = {}
diff = set()
def hash_diff(file_path):
    print_py_file_paths(file_path)

    for f in fList:
        #dic.update(hash_functions_in_file(f))
        res = hash_functions_in_file(f)
        for k, v in res.items():
                if k in dic.keys() and v != dic[k]:
                    diff.add(k)
                else:
                    dic[k] = v

s = time.time()

hash_diff('C:\\Users\\user\\Downloads\\CAD_SPL-master\\ver5_CAD_MODE')

#hash_diff('C:\\Users\\user\\Downloads\\CAD_SPL-master\\ver1_BASE_MODE')
#hash_diff('C:\\Users\\user\\Downloads\\CAD_SPL-master\\ver2_ToF_MODE')
#hash_diff('C:\\Users\\user\\Downloads\\CAD_SPL-master\\ver3_CAMERA_MODE')
#hash_diff('C:\\Users\\user\\Downloads\\CAD_SPL-master\\ver4_YOLO_MODE')

print(dic)
print(diff)
e = time.time()
print(e - s)

{'run5_CAD.py\\__init__()': '794cf82b8e5f0e4e8e86242695ea675cf1b7680a857bcbf219e73fee87036473', 'Actor.py\\take_cmd_from_planner()': '85b86a1725da8891dbc68d1bc46ec9773b5e442872a96d99c32e3ec214793f44', 'Actor.py\\change_cmd_is_safe()': '0a7e8373f90b3263c40ba6da36d1c39705d4a674efd6e5584752add585242a21', 'Actor.py\\change_cmd_for_drone()': 'e7da42f7169f3dc556c8012d24601afd4d80469a9d09493418e408bf16009b67', 'Actor.py\\send_to_actuator()': 'fb5914697c2956e04705983e07d60daadf303621c5e5607e36689dd89404c501', 'ObjectDetector.py\\detect_from_frame()': 'ff6e832d26948def94360f55d963ac6e31517ffc98af5de605d91e3adeeb04a9', 'Sensor.py\\take_data_from_sensor()': '748268ac76b0db6a3239cebc0555eedc0623189b3e145bc354e31b5d4ff1a778', 'Sensor.py\\change_data_to_info()': 'fd01bfd430805eaea8544485fc2e0c6ec6d4b9a7b8762c1642a302bd98b76a5f', 'Sensor.py\\save_to_planner()': 'a80e34614fb5c091db07e8e1d3d6ce236fc4fb201741c9be1b22936b2a910eac', 'ValueChanger.py\\change_mm_to_cm()': '0e36002b47dff37a6be1de864179fb011c

In [2]:
## Preparation phase (instrumentation) ##
import os
import ast
import time


def getAllFilePathsFromDir(folder_path):
    fList = list()
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                fList.append(file_path)
    return fList

def instrument(file_path):
    with open(file_path, "rt", encoding="UTF-8") as f:
        code = f.read()
        tree = ast.parse(code)
        
        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef):
                # Get the function name
                function_name = node.name
                # Create a new print statement node
                print_node = ast.parse(f'print("Function Name:", "{function_name}")').body[0]
                # Insert the print statement at the beginning of the function body
                node.body.insert(0, print_node)
        
    return ast.unparse(tree)

s = time.time()
paths = getAllFilePathsFromDir('C:\\Users\\user\\Downloads\\CAD_SPL-master\\ver5_CAD_MODE')

for f in paths:
    new_code = instrument(f)
    print(new_code)

e = time.time()
print(e-s)

import threading
import sys
import socket
from CAD.Plan.Planner5 import Planner
from CAD.Tello.Tello8889Sensor import Tello8889Sensor
from CAD.Tello.Tello11111Sensor import Tello11111Sensor
from CAD.Tello.Tello8889Actor import Tello8889Actor
from CAD.Test.TelloVirtualController import TelloVirtualController
"\n- Architecture: Sense - Plan - Act Pattern\n\n- 입력방식(유선, 무선 및 포트)에 따라 Sense 계열 클래스 생성\n- 연산을 담당하는 Planner 클래스 생성\n- 출력방식(유선, 무선 및 포트)에 따라 Act 계열 클래스 생성\n\n- Sense 계열 클래스들은 \n    1) 데이터를 가져오고'\n    2) 데이터를 Planner가 받아들일 수 있는 정보로 변경하고V2\n    3) Planner에 저장\n    \n- Planner 클래스는\n    1) 저장된 값을 원하는 정보로 가공(3차원 좌표값으로 변경)하고\n    2) 가공한 정보를 바탕으로 회피 명령을 생성하고  n /\n    3) 생성된 명령을 Planner 내부에 저장\n    \n- Act 계열 클래스들은 \n    1) Planner에 저장된 값을 가져와서\n    2) Drone이 이해할 수 있는 값으로 변경하고\n    2) Actuator로 전송\n\n- 이 모든 과정은 순차적이 아닌 병렬적으로 수행(사용자 명령이 존재하기 때문)\n"

class Main:

    def __init__(self):
        print('Function Name:', '__init__')
        print('>>> 프로그램 준비중...')
        self.stop_event = th

In [1]:
## Selection phase (Preprocessing) ##

import os
import inspect
import hashlib
import time

fList = list()


def print_py_file_paths(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.py') :
                file_path = os.path.join(root, file)
                fList.append(file_path)
    print(len(fList))


def hash_function(source_code):
    """
    Takes in the source code of a function as a string and returns the SHA-256 hash
    of the source code as a hexadecimal string.
    """
    hash_object = hashlib.sha256(source_code.encode())
    hex_dig = hash_object.hexdigest()
    return hex_dig


def hash_functions_in_file(file_path):
    """
    Takes in a file path and returns a dictionary mapping function names to their
    corresponding SHA-256 hash of source code.
    """
    function_hashes = {}

    with open(file_path, "rt", encoding="UTF-8") as f:
        source_code = f.read()
        file_name = file_path.split("\\")[-1]
        file_name = file_name.split(".")[0]
        # Split source code by "def " to get each function's source code
        function_sources = source_code.split("def ")[1:]

        for function_source in function_sources:
            # Get the function name and source code
            function_name = function_source.split("(")[0]
            source_code = "def " + function_source

            # Hash the source code and store in the dictionary
            function_hash = hash_function(source_code)
            function_hashes[file_name + "\\" + function_name + "()"] = function_hash

    return function_hashes


s = time.time()
print_py_file_paths('C:\\Users\\user\\Downloads\\CAD_SPL-master\\ver5_CAD_MODE')

dic = dict()
for f in fList:
    dic.update(hash_functions_in_file(f))

print(dic)

e = time.time()
print(e - s)

21
{'run5_CAD\\__init__()': '794cf82b8e5f0e4e8e86242695ea675cf1b7680a857bcbf219e73fee87036473', 'Actor\\take_cmd_from_planner()': '85b86a1725da8891dbc68d1bc46ec9773b5e442872a96d99c32e3ec214793f44', 'Actor\\change_cmd_is_safe()': '0a7e8373f90b3263c40ba6da36d1c39705d4a674efd6e5584752add585242a21', 'Actor\\change_cmd_for_drone()': 'e7da42f7169f3dc556c8012d24601afd4d80469a9d09493418e408bf16009b67', 'Actor\\send_to_actuator()': 'fb5914697c2956e04705983e07d60daadf303621c5e5607e36689dd89404c501', 'ObjectDetector\\detect_from_frame()': 'ff6e832d26948def94360f55d963ac6e31517ffc98af5de605d91e3adeeb04a9', 'Sensor\\take_data_from_sensor()': '748268ac76b0db6a3239cebc0555eedc0623189b3e145bc354e31b5d4ff1a778', 'Sensor\\change_data_to_info()': 'fd01bfd430805eaea8544485fc2e0c6ec6d4b9a7b8762c1642a302bd98b76a5f', 'Sensor\\save_to_planner()': 'a80e34614fb5c091db07e8e1d3d6ce236fc4fb201741c9be1b22936b2a910eac', 'ValueChanger\\change_mm_to_cm()': '0e36002b47dff37a6be1de864179fb011c539243afc960812f71ab1d84f31

In [96]:
# Selection phase (Test Case Selection)

# automatically obtained hash values
hDict = {'t1001':['89d21e694ca58e764db12b0d727b531e78e03655495e1ebb9f94bbbef0708be13'], 't1002':['3fa70809476707dee1f9fc4c2f37eed27fdb969c2f3ebdfb12d4f7c8055221e0'], 't1003':['d8b14d7878da2a82378d2d6d790c01d9d206bb13e176376602ff0347e4e75095'], 't1004':['9f51f1b63e310d0cff14c2a3512bacd79de071850392009705a30db7642664c2'], 't1005':['9be770fe5e98fea4a379a13b66cd1c36165d930cc5a6a1584e9bac625ff93d81'], 't1006':['14c30960c8b17993c7da54991af62b79c270b073f0075e58af1d01f4de6ab23d'], 't1007':['89d21394ca58e764db12b0d727b531e78e03655495e1ebb9f94bbbef0708be13'], 't1008':['0180ae596dd49cbf642b5fa73118a1ff9b59d6d030a37473d58f74c6cafccc7f'], 't1009':['6d099b24c6c8573197ee23f5a490a430fea49e7d6912c8bfbf5705d6534ac348'], 
         't1010':['3fa70809476707dee1f9fc4c2f37eed27fdb969c2f3ebdfb12d4f7c8055221e0'], 't1011':['9f51f1b63e310d0cff14c2a3512bacd79de071850392009705a30db7642664c2'], 't1012':['9be770fe5e98fea4a379a13b66cd1c36165d930cc5a6a1584e9bac625ff93d81'], 't1013':['14c30960c8b17993c7da54991af62b79c270b073f0075e58af1d01f4de6ab23d'], 't1014':['89d21394ca58e764db12b0d727b531e78e03655495e1ebb9f94bbbef0708be13'], 't1015':['0180ae596dd49cbf642b5fa73118a1ff9b59d6d030a37473d58f74c6cafccc7f'], 't1021':['64faae596dd49cbf642565f73118a1469b59d6d030a37473d58f74c6cafc3443'], 't1022':['6d099b24c6c8573197ee23f5a490a430fea49e7d6912c8bfbf5705d6534ac348'], 't1023':['557f384c08c6517e013304cc0501a36b9f615489b072aaae3d3ef2d4fb00faf3'], 
         't1024':['f3a973adb5aeb504adea824eda7b324ecab7f9d8050ff224aac9285b9212a98a'], 't1025':['7cc3e9b6d7773f707a7929c0b924bb2f619b18ed6723b39def5bcac99484ec9c'], 't1026':['dd92214ff49d5dde577669bed0439a05255d9ab324f116ea870c9aff8eb1b373'], 't1027':['c837f48496fe49c2b40c5c0c8c4d05b68d3128a745446877ad887a1f9bc5a5cf'], 't1028':['5ea52a77cadca7dcad5981cf6a4013e43087965023efdac5108e4e8fbb537ea8'], 't1029':['0e36002b47dff37a6be1de864179fb011c539243afc960812f71ab1d84f315cd'], 't1030':['24d52ce18954f784cada1bf57484dd2a270403fbdd0b62ecc66bb1823e1436f5'], 't1031':['27b790699327188b6118b9fc74fc9e15083e8af06128abdd06afc1ba168d1113'], 't1032':['b11acaa010a45855494a1a4bb00665797767c34320c8a077fef569c82954fee9'], 
         't2001':['64ddae596dd49cbf642565f73118a1469b59d6d030a37473d58f74c6cafc3443'], 't2002':['6d099b24c6c8573197e553f5a490a430fea49e7d6912c8bfbf5705d6534ac348'], 't2003':['a6181a537629c3eb3ff0d8f69296b299603cdffaac9aa7b4b6b8431d9f89836f'], 't2004':['42af89feb61376642de588eb68dd947878de64e05fa3c8b68d516943ba85766a'], 't3001':['64fa33596dd49cbf642565f73118a1469b59d6d030a37473d58f74c6cafc3443'], 't3002':['7367a5a5e85ab6da9a3fee7cbb0a75ce0842643263b90a8170a263a03ae6a633'], 't3003':['d786b6a9cfe354db661204cb14e1f6c539eaddca70cdd3beb0c54f61382954a1'], 't3004':['9a3fe3d2f83bdc97147e802acb8913354735c2777999c497e1548b0b639d3765'], 't3005':['0e36002b47dff37a6be1de864179fb011c5aa243afc960812f71ab1d84f315cd'], 
         't3006':['7266386e98c21a363846032c74a443aab5d1ad97a65f07e1d91c6ca8be789122'], 't3007':['1c09a1698cc13773b3e2faff712405f13f633ff943cb2325bb45d658767bfe8c'], 't3008':['438a2b69acd8350590e528b709782892094ddbf3cbe5be0466f63773a4c10d9c'], 't3009':['252e5cb29deb35fbf611c16e1cb30932f0ab6cd2379b24949ce1c7cafa7ce244'], 't3010':['82a523c8cdd1f335c3ad6aa276865b29f9013d1534bf632d66f91ee1e1834805'], 't4001':['1a9996023a3cc1e2f4811c04678f2ef7c5bf56bbf6713383bb8712326194cecd'], 't4002':['382b82ffa1058b6915b754bef2bf86a67c5c78de146abd2b2346e946e9cc5076'], 't4003':['d786b6a9cfe354db661ee4cb14e1f6c539eaddca70cdd3beb0c54f61382954a1'], 't4004':['9a3fe3d2f83bdc97147e802acb8913354115c2777999c497e1548b0b639d3765'], 
         't4005':['382b82ffa1058b6915b754bef2bf86a67c5c78de146abd2b2346e946e9cc5076'], 't4006':['4159dbbc723bd962ffc6befda35990a28f9b996e09e83093a724cc57107367c1']}
s = time.time()
sel = []
for k, v in hDict.items():
    if v[0] in dic.values():
        sel.append(k)
        print(k)

e = time.time()
print(e-s)

t1002
t1004
t1005
t1006
t1007
t1008
t1009
t1010
t1011
t1012
t1013
t1014
t1015
t1022
t1023
t1024
t1025
t1026
t1027
t1028
t1029
t1030
t1031
t1032
t2003
t2004
t3002
t3007
t3008
t3009
t3010
t4001
t4002
t4005
t4006
0.000997304916381836


In [100]:
# Preparation (TSTM update)
s = time.time()
for k, v in hDict.items():
    v.append("P5")

TC4CA = ["t5001","t5002","t5003","t5004","t5005","t5006"]
hashes = [["64ddae596dd49cbf642565f73118a1469b59d6d030a37473d58f74c6cafc3443"], ["7cc3e9b6d7773f707a7929c0b924bb2f619b18ed6723b39def5bcac99484ec9c"], ["3fa70809476707dee1f9fc4c2f37eed27fdb969c2f3ebdfb12d4f7c8055221e0"], ["9f51f1b63e310d0cff14c2a3512bacd79de071850392009705a30db7642664c2"], ["42af89feb61376642de588eb68dd947878de64e05fa3c8b68d516943ba85766a"], ["7367a5a5e85ab6da9a3fee7cbb0a75ce0842643263b90a8170a263a03ae6a633"]]
for i in range(6):
    hDict[TC4CA[i]] = hashes[i]
    
print(hDict)
e = time.time()
print(e-s)

{'t1001': ['89d21e694ca58e764db12b0d727b531e78e03655495e1ebb9f94bbbef0708be13', 'P5'], 't1002': ['3fa70809476707dee1f9fc4c2f37eed27fdb969c2f3ebdfb12d4f7c8055221e0', 'P5'], 't1003': ['d8b14d7878da2a82378d2d6d790c01d9d206bb13e176376602ff0347e4e75095', 'P5'], 't1004': ['9f51f1b63e310d0cff14c2a3512bacd79de071850392009705a30db7642664c2', 'P5'], 't1005': ['9be770fe5e98fea4a379a13b66cd1c36165d930cc5a6a1584e9bac625ff93d81', 'P5'], 't1006': ['14c30960c8b17993c7da54991af62b79c270b073f0075e58af1d01f4de6ab23d', 'P5'], 't1007': ['89d21394ca58e764db12b0d727b531e78e03655495e1ebb9f94bbbef0708be13', 'P5'], 't1008': ['0180ae596dd49cbf642b5fa73118a1ff9b59d6d030a37473d58f74c6cafccc7f', 'P5'], 't1009': ['6d099b24c6c8573197ee23f5a490a430fea49e7d6912c8bfbf5705d6534ac348', 'P5'], 't1010': ['3fa70809476707dee1f9fc4c2f37eed27fdb969c2f3ebdfb12d4f7c8055221e0', 'P5'], 't1011': ['9f51f1b63e310d0cff14c2a3512bacd79de071850392009705a30db7642664c2', 'P5'], 't1012': ['9be770fe5e98fea4a379a13b66cd1c36165d930cc5a6a1584e9b